In [1]:
import sys; sys.path.append('../'); sys.path.append('../Preprocess/')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression

# use sklearn metrics, single function
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import time
from Preprocess.preprocess2 import *
from imblearn.over_sampling import SMOTE

In [2]:
applySmote = False

In [3]:
preprocessor = Preprocessor()
df_h = preprocessor.preprocess()
df_h.head()

,Age,Height,Weight,Veg_Consump,Water_Consump,Meal_Count,Phys_Act,Time_E_Dev,Gender_Female,Gender_Male,...,Fam_Hist_no,Fam_Hist_yes,H_Cal_Burn_no,H_Cal_Burn_yes,Transport_Automobile,Transport_Bike,Transport_Motorbike,Transport_Public_Transportation,Transport_Walking,Body_Level
0,22.547298,1.722461,51.881263,2.663421,1.041110,3.000000,0.794402,1.391948,1,0,...,0,1,1,0,0,0,0,1,0,0
1,19.799054,1.743702,54.927529,2.000000,2.847264,3.289260,1.680844,2.000000,0,1,...,0,1,1,0,0,0,0,1,0,0
2,17.823438,1.708406,50.000000,1.642241,1.099231,3.452590,0.418875,1.000000,1,0,...,1,0,1,0,0,0,0,1,0,0
3,19.007177,1.690727,49.895716,1.212908,1.029703,3.207071,2.000000,1.000000,1,0,...,1,0,1,0,0,0,0,1,0,0
4,19.729250,1.793315,58.195150,2.508835,2.076933,3.435905,2.026668,1.443328,0,1,...,0,1,1,0,1,0,0,0,0,0


In [107]:
df_h = pd.read_csv('../dataset/body_level_classification_train.csv')
df_h.head()

,Gender,Age,Height,Weight,H_Cal_Consump,Veg_Consump,Water_Consump,Alcohol_Consump,Smoking,Meal_Count,Food_Between_Meals,Fam_Hist,H_Cal_Burn,Phys_Act,Time_E_Dev,Transport,Body_Level
0,Female,22.547298,1.722461,51.881263,yes,2.663421,1.041110,no,no,3.000000,Frequently,yes,no,0.794402,1.391948,Public_Transportation,Body Level 1
1,Male,19.799054,1.743702,54.927529,yes,2.000000,2.847264,Sometimes,no,3.289260,Sometimes,yes,no,1.680844,2.000000,Public_Transportation,Body Level 1
2,Female,17.823438,1.708406,50.000000,yes,1.642241,1.099231,Sometimes,no,3.452590,Sometimes,no,no,0.418875,1.000000,Public_Transportation,Body Level 1
3,Female,19.007177,1.690727,49.895716,yes,1.212908,1.029703,Sometimes,no,3.207071,Sometimes,no,no,2.000000,1.000000,Public_Transportation,Body Level 1
4,Male,19.729250,1.793315,58.195150,yes,2.508835,2.076933,no,no,3.435905,Sometimes,yes,no,2.026668,1.443328,Automobile,Body Level 1


In [108]:
df_h.head()

,Gender,Age,Height,Weight,H_Cal_Consump,Veg_Consump,Water_Consump,Alcohol_Consump,Smoking,Meal_Count,Food_Between_Meals,Fam_Hist,H_Cal_Burn,Phys_Act,Time_E_Dev,Transport,Body_Level
0,Female,22.547298,1.722461,51.881263,yes,2.663421,1.041110,no,no,3.000000,Frequently,yes,no,0.794402,1.391948,Public_Transportation,Body Level 1
1,Male,19.799054,1.743702,54.927529,yes,2.000000,2.847264,Sometimes,no,3.289260,Sometimes,yes,no,1.680844,2.000000,Public_Transportation,Body Level 1
2,Female,17.823438,1.708406,50.000000,yes,1.642241,1.099231,Sometimes,no,3.452590,Sometimes,no,no,0.418875,1.000000,Public_Transportation,Body Level 1
3,Female,19.007177,1.690727,49.895716,yes,1.212908,1.029703,Sometimes,no,3.207071,Sometimes,no,no,2.000000,1.000000,Public_Transportation,Body Level 1
4,Male,19.729250,1.793315,58.195150,yes,2.508835,2.076933,no,no,3.435905,Sometimes,yes,no,2.026668,1.443328,Automobile,Body Level 1


# XGBOOST

In [93]:
# XGBoost
import xgboost as xgb

X = df_h.iloc[:, :-1]
y = df_h.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBClassifier(
                            booster='gbtree',
                            learning_rate=0.3,
                            max_depth=3,
                            objective="multi:softprob",
                            random_state=42,
                            num_class=4,
                            # eval_metric="auc",
                            eval_metric="mlogloss",
                            )
xgb_model.fit(X_train, y_train)

y_pred_train = xgb_model.predict(X_train)
y_pred = xgb_model.predict(X_test)

print('Training Metrics for XGBoost:')
# print(classification_report(y_train, y_pred_train, digits=4))
print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

# print(classification_report(y_train, y_pred_train, digits=4).split('\n')[-2])
# print(classification_report(y_test, y_pred, digits=4).split('\n')[-2])

print('\nTesting Metrics for XGBoost:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for XGBoost:
[[159   0   0   0]
 [  0 156   0   0]
 [  0   0 324   0]
 [  0   0   0 542]]

Testing Metrics for XGBoost:
              precision    recall  f1-score   support

           0     0.9394    1.0000    0.9688        31
           1     0.9762    0.9111    0.9425        45
           2     0.9750    0.9512    0.9630        82
           3     0.9787    1.0000    0.9892       138

    accuracy                         0.9730       296
   macro avg     0.9673    0.9656    0.9659       296
weighted avg     0.9732    0.9730    0.9727       296

[[ 31   0   0   0]
 [  2  41   2   0]
 [  0   1  78   3]
 [  0   0   0 138]]


# LightGBM

In [113]:
# lightgbm
from lightgbm.sklearn import LGBMClassifier
    
lgb_model = LGBMClassifier(
    boosting_type='gbdt',
    class_weight=None,
    # importance_type='split',
    learning_rate=0.2,
    # max_depth=-1,
    # min_child_samples=20,
    # min_child_weight=0.001,
    # min_split_gain=0.0,
    n_estimators=1000,
    # num_leaves=31,
    objective='multiclass',
    # reg_alpha=0.0,
    # reg_lambda=0.,
    # verbose=-10
)

# lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

y_pred_train = lgb_model.predict(X_train)
y_pred = lgb_model.predict(X_test)

# print('Training Metrics for LightGBM:')
# print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print(classification_report(y_train, y_pred_train, digits=4).split('\n')[-2])
print(classification_report(y_test, y_pred, digits=4).split('\n')[-2])

print('\nTesting Metrics for LightGBM:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, normalize='true'))

weighted avg     1.0000    1.0000    1.0000      1181
weighted avg     0.9769    0.9764    0.9761       296

Testing Metrics for LightGBM:
              precision    recall  f1-score   support

           0     0.9394    1.0000    0.9688        31
           1     1.0000    0.9111    0.9535        45
           2     0.9753    0.9634    0.9693        82
           3     0.9787    1.0000    0.9892       138

    accuracy                         0.9764       296
   macro avg     0.9734    0.9686    0.9702       296
weighted avg     0.9769    0.9764    0.9761       296

[[ 31   0   0   0]
 [  2  41   2   0]
 [  0   0  79   3]
 [  0   0   0 138]]
[[1.         0.         0.         0.        ]
 [0.04444444 0.91111111 0.04444444 0.        ]
 [0.         0.         0.96341463 0.03658537]
 [0.         0.         0.         1.        ]]


# Looks like SVM Overfits, specially with high C

In [6]:
# SVM
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

if applySmote:
    print('Before SMOTE: ', np.bincount(y_train))
    smote = SMOTE(random_state=0)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # check the number of samples per class
    print('After SMOTE: ', np.bincount(y_train_smote))
    print()
    X_train = X_train_smote
    y_train = y_train_smote

# SVM
# svm = SVC(kernel='linear', C=1.0, random_state=0)
svm = SVC(kernel='linear', C= 100, random_state=0)
svm.fit(X_train, y_train)

y_pred_train = svm.predict(X_train)
y_pred = svm.predict(X_test)

print('Training Metrics for SVM:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for SVM:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for SVM:
              precision    recall  f1-score   support

           0     0.9803    0.9933    0.9868       150
           1     0.9873    0.9750    0.9811       160
           2     0.9969    0.9937    0.9953       319
           3     0.9982    1.0000    0.9991       552

    accuracy                         0.9941      1181
   macro avg     0.9907    0.9905    0.9906      1181
weighted avg     0.9941    0.9941    0.9941      1181


Testing Metrics for SVM:
              precision    recall  f1-score   support

           0     0.9524    1.0000    0.9756        40
           1     0.9744    0.9268    0.9500        41
           2     0.9882    0.9655    0.9767        87
           3     0.9846    1.0000    0.9922       128

    accuracy                         0.9797       296
   macro avg     0.9749    0.9731    0.9737       296
weighted avg     0.9799    0.9797    0.9796       296

[[ 40   0   0   0]
 [  2  38   1   0]
 [  0   1  84   2]
 [  0   0   0 128]]


# Linear Regression

In [7]:
# linear regression
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# applySmote = True
if applySmote:
    print('Before SMOTE: ', np.bincount(y_train))
    smote = SMOTE(random_state=0)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # check the number of samples per class
    print('After SMOTE: ', np.bincount(y_train_smote))
    print()
    X_train = X_train_smote
    y_train = y_train_smote

# linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = np.round(lr.predict(X_train))
y_pred = np.round(lr.predict(X_test))

print('Training Metrics for Linear Regression:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for Linear Regression:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for Linear Regression:
              precision    recall  f1-score   support

        -1.0     0.0000    0.0000    0.0000         0
         0.0     0.9182    0.6733    0.7769       150
         1.0     0.6029    0.7875    0.6829       160
         2.0     0.7514    0.8715    0.8070       319
         3.0     0.9873    0.8442    0.9102       552
         4.0     0.0000    0.0000    0.0000         0

    accuracy                         0.8222      1181
   macro avg     0.5433    0.5294    0.5295      1181
weighted avg     0.8627    0.8222    0.8346      1181


Testing Metrics for Linear Regression:
              precision    recall  f1-score   support

         0.0     0.8710    0.6750    0.7606        40
         1.0     0.5536    0.7561    0.6392        41
         2.0     0.7228    0.8391    0.7766        87
         3.0     0.9811    0.8125    0.8889       128
         4.0     0.0000    0.0000    0.0000         0

    accuracy                         0.7939       2

c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classificat

# Logistic Regression

In [8]:
# logistic regression, we have 4 classes
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

if applySmote:
    print('Before SMOTE: ', np.bincount(y_train))
    smote = SMOTE(random_state=0)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # check the number of samples per class
    print('After SMOTE: ', np.bincount(y_train_smote))
    print()
    X_train = X_train_smote
    y_train = y_train_smote


# logistic regression
lr = LogisticRegression( solver='lbfgs', multi_class='multinomial', max_iter=5000, random_state=0, penalty='l2')
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred = lr.predict(X_test)

print('Training Metrics for Logistic Reg:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for Logistic Reg:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for Logistic Reg:
              precision    recall  f1-score   support

           0     0.8659    0.9467    0.9045       150
           1     0.8222    0.6937    0.7525       160
           2     0.8529    0.8903    0.8712       319
           3     0.9654    0.9601    0.9628       552

    accuracy                         0.9035      1181
   macro avg     0.8766    0.8727    0.8727      1181
weighted avg     0.9030    0.9035    0.9021      1181


Testing Metrics for Logistic Reg:
              precision    recall  f1-score   support

           0     0.7959    0.9750    0.8764        40
           1     0.8000    0.6829    0.7368        41
           2     0.8659    0.8161    0.8402        87
           3     0.9231    0.9375    0.9302       128

    accuracy                         0.8716       296
   macro avg     0.8462    0.8529    0.8459       296
weighted avg     0.8720    0.8716    0.8697       296

[[ 39   1   0   0]
 [ 10  28   3   0]
 [  0   6  71  10]
 [ 

# SVM with RBF Kernel

In [9]:
# SVM
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

if applySmote:
    print('Before SMOTE: ', np.bincount(y_train))
    smote = SMOTE(random_state=0)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # check the number of samples per class
    print('After SMOTE: ', np.bincount(y_train_smote))
    print()
    X_train = X_train_smote
    y_train = y_train_smote
    
# SVM
svm = SVC(kernel='rbf', C=1.0, random_state=0)
svm.fit(X_train, y_train)

y_pred_train = svm.predict(X_train)
y_pred = svm.predict(X_test)

print('Training Metrics for SVM with RBF Kernel:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for SVM with RBF Kernel:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for SVM with RBF Kernel:
              precision    recall  f1-score   support

           0     0.7267    0.8333    0.7764       150
           1     0.5636    0.3875    0.4593       160
           2     0.6513    0.8433    0.7350       319
           3     0.9527    0.8388    0.8921       552

    accuracy                         0.7782      1181
   macro avg     0.7236    0.7257    0.7157      1181
weighted avg     0.7899    0.7782    0.7763      1181


Testing Metrics for SVM with RBF Kernel:
              precision    recall  f1-score   support

           0     0.6522    0.7500    0.6977        40
           1     0.4412    0.3659    0.4000        41
           2     0.6283    0.8161    0.7100        87
           3     0.9417    0.7578    0.8398       128

    accuracy                         0.7196       296
   macro avg     0.6659    0.6724    0.6619       296
weighted avg     0.7412    0.7196    0.7215       296

[[30 10  0  0]
 [15 15 11  0]
 [ 1  9 71  6]
 

# Naive Bayes

In [10]:
# Naive Bayes
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

if applySmote:
    print('Before SMOTE: ', np.bincount(y_train))
    smote = SMOTE(random_state=0)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # check the number of samples per class
    print('After SMOTE: ', np.bincount(y_train_smote))
    print()
    X_train = X_train_smote
    y_train = y_train_smote
    
# Naive Bayes
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = MultinomialNB()
# nb = GaussianNB( )
# nb = BernoulliNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
y_pred_train = nb.predict(X_train)

print('Training Metrics for Naive Bayes:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for Naive Bayes:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for Naive Bayes:
              precision    recall  f1-score   support

           0     0.7415    0.7267    0.7340       150
           1     0.6952    0.4562    0.5509       160
           2     0.5697    0.6019    0.5854       319
           3     0.8209    0.8804    0.8497       552

    accuracy                         0.7282      1181
   macro avg     0.7069    0.6663    0.6800      1181
weighted avg     0.7260    0.7282    0.7231      1181


Testing Metrics for Naive Bayes:
              precision    recall  f1-score   support

           0     0.6216    0.5750    0.5974        40
           1     0.5172    0.3659    0.4286        41
           2     0.5217    0.5517    0.5363        87
           3     0.7899    0.8516    0.8195       128

    accuracy                         0.6588       296
   macro avg     0.6126    0.5860    0.5955       296
weighted avg     0.6506    0.6588    0.6521       296

[[ 23   2  15   0]
 [ 14  15  11   1]
 [  0  11  48  28]
 [  0

# SVM linear with SMOTE

In [11]:
X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# apply smote
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


# check the number of samples per class
print('Before SMOTE: ', np.bincount(y_train))
print('After SMOTE: ', np.bincount(y_train_smote))

print()

# SVM, linear, C=10
svm = SVC(kernel='linear', C=100, random_state=0)
svm.fit(X_train_smote, y_train_smote)

y_pred = svm.predict(X_test)
y_pred_train = svm.predict(X_train)

print('Training Metrics for SVM linear with SMOTE:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for SVM linear with SMOTE:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Before SMOTE:  [150 160 319 552]
After SMOTE:  [552 552 552 552]

Training Metrics for SVM linear with SMOTE:
              precision    recall  f1-score   support

           0     0.9934    1.0000    0.9967       150
           1     0.9876    0.9938    0.9907       160
           2     0.9969    0.9937    0.9953       319
           3     1.0000    0.9982    0.9991       552

    accuracy                         0.9966      1181
   macro avg     0.9945    0.9964    0.9954      1181
weighted avg     0.9966    0.9966    0.9966      1181


Testing Metrics for SVM linear with SMOTE:
              precision    recall  f1-score   support

           0     0.9091    1.0000    0.9524        40
           1     0.9730    0.8780    0.9231        41
           2     0.9655    0.9655    0.9655        87
           3     0.9844    0.9844    0.9844       128

    accuracy                         0.9662       296
   macro avg     0.9580    0.9570    0.9563       296
weighted avg     0.9671    0.96

# Catboost 

In [12]:
from catboost import CatBoostClassifier

X = df_h.iloc[:, :-1].values
y = df_h.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create the CatBoost model
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

# Fit the model on the training data
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred = model.predict(X_test)

print('Training Metrics for Catboost:')
print(classification_report(y_train, y_pred_train, digits=4))
# print(confusion_matrix(y_train, y_pred_train))
# print(confusion_matrix(y_train, y_pred_train, normalize='true'))

print('\nTesting Metrics for Catboost:')
print(classification_report(y_test, y_pred, digits=4))
print(confusion_matrix(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred, normalize='true'))

Training Metrics for Catboost:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       150
           1     1.0000    0.9812    0.9905       160
           2     0.9907    0.9969    0.9938       319
           3     0.9982    1.0000    0.9991       552

    accuracy                         0.9966      1181
   macro avg     0.9972    0.9945    0.9958      1181
weighted avg     0.9966    0.9966    0.9966      1181


Testing Metrics for Catboost:
              precision    recall  f1-score   support

           0     0.9750    0.9750    0.9750        40
           1     0.9474    0.8780    0.9114        41
           2     0.9419    0.9310    0.9364        87
           3     0.9621    0.9922    0.9769       128

    accuracy                         0.9561       296
   macro avg     0.9566    0.9441    0.9499       296
weighted avg     0.9559    0.9561    0.9557       296

[[ 39   1   0   0]
 [  1  36   4   0]
 [  0   1  81   5]
 [  0   0  

In [13]:
# Create a dictionary with sample data
new_data_dict = {
    'Age': [32, 45, 21],
    'Height': [1.65, 1.75, 1.68],
    'Weight': [72, 89, 55],
    'Veg_Consump': [3, 2, 4],
    'Water_Consump': [4, 3, 2],
    'Meal_Count': [3, 4, 2],
    'Phys_Act': [2, 3, 4],
    'Time_E_Dev': [3, 2, 1],
    'Gender_Female': [0, 1, 0],
    'Gender_Male': [1, 0, 1],
    'H_Cal_Consump_no': [1, 0, 0],
    'H_Cal_Consump_yes': [0, 1, 1],
    'Alcohol_Consump_Always': [0, 0, 1],
    'Alcohol_Consump_Frequently': [1, 0, 0],
    'Alcohol_Consump_Sometimes': [0, 1, 0],
    'Alcohol_Consump_no': [0, 0, 0],
    'Smoking_no': [1, 1, 0],
    'Smoking_yes': [0, 0, 1],
    'Food_Between_Meals_Always': [0, 1, 0],
    'Food_Between_Meals_Frequently': [0, 0, 1],
    'Food_Between_Meals_Sometimes': [1, 0, 0],
    'Food_Between_Meals_no': [0, 0, 0],
    'Fam_Hist_no': [1, 0, 1],
    'Fam_Hist_yes': [0, 1, 0],
    'H_Cal_Burn_no': [1, 0, 0],
    'H_Cal_Burn_yes': [0, 1, 1],
    'Transport_Automobile': [1, 0, 0],
    'Transport_Bike': [0, 1, 0],
    'Transport_Motorbike': [0, 0, 1],
    'Transport_Public_Transportation': [0, 0, 0],
    'Transport_Walking': [0, 0, 0]
}

# Convert the dictionary to a Pandas DataFrame
new_data = pd.DataFrame(new_data_dict)

print(len(df_h.columns))
print("---------------------------")
print(len(new_data.columns))
print("")

# Make prediction using the model
predictions = model.predict(new_data)

# Print the predicted class labels
print("predictions are : \n", predictions)


32
---------------------------
31

predictions are : 
 [[2]
 [2]
 [1]]
